<a href="https://colab.research.google.com/github/TheBlackRus/liveproject_HumanPoseEstimation/blob/master/liveproject_humanpose_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to part 2. Training of a CNN wiht pytorch

In [1]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat


--2020-05-02 13:49:51--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  37.1MB/s    in 6.5s    

2020-05-02 13:49:57 (26.7 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2020-05-02 13:49:58--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  17.6MB/s    in 3.5s    

2020-05-02 13:50:02 (17.6 MB/s) - ‘test_32x32.mat’ saved [64275384/64

(32, 32, 3, 73257)
(73257, 1)


In [0]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from scipy.io import loadmat
from matplotlib.pyplot import imshow

In [0]:
from torch.utils.data import Dataset
class SVHN_dataset(Dataset):

    def __init__(self, data):
        # Transform data to Torch Tensors
        self.images = torch.tensor(data['X']).permute([3,2,0,1])
        self.labels = torch.tensor(data['y'])
        self.size = self.labels.shape[0]

        # replace label 10 with label 0
        self.labels[np.where(self.labels == 10)] = 0

        # convert to float and normalize images to 0..1 range
        self.images = self.images.to(dtype=torch.float)
        self.images = self.images / 255.0

        self.labels = self.labels.to(dtype=torch.float)


    
    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [0]:
train_data = loadmat('train_32x32.mat')
test_data = loadmat('test_32x32.mat')

In [0]:
train_dataset = SVHN_dataset(train_data)

Values are supposed to be normalized

In [29]:
print("images range [", train_dataset.images.min(),",", train_dataset.images.max(),"]",train_dataset.images.shape)
print("labels",train_dataset.labels.unique())

images range [ tensor(0.) , tensor(1.) ] torch.Size([73257, 3, 32, 32])
labels tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])


Creating the net
1. convolution, kernel_size=5, channels=6, stride=1, padding=2
2. batch-normalization
3. ReLU
4. Max-pool, kernel_size=2, stride=2

5. convolution, kernel_size=3, channels=12, stride=1, padding=1
6. batch-normalization
7. ReLU
8. Max-pool, kernel_size=2, stride=2

9. convolution, kernel_size=3, channels=24, stride=1, padding=1
10. batch-normalization
11. ReLU
12. Max-pool, kernel_size=2, stride=2

13. fully connected layer, output_size=10

In [0]:
import torch.nn as nn
import torch.nn.functional as F


In [0]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #block
        self.conv_1 = nn.Conv2d(in_channels = 3, out_channels=6,kernel_size=5,stride=1,padding=2)
        self.batch_norm_1 = nn.BatchNorm2d(6) # same as out_channels before layer 
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(2, 2)


        self.conv_2 = nn.Conv2d(in_channels = 6, out_channels=12,kernel_size=3,stride=1,padding=1)
        self.batch_norm_2 = nn.BatchNorm2d(12) # same as out_channels before layer 
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.conv_3 = nn.Conv2d(in_channels= 12, out_channels=24,kernel_size=3,stride=1,padding=1)
        self.batch_norm_3 = nn.BatchNorm2d(24) # same as out_channels before layer 
        self.relu_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(24 * 4 * 4, 10)# 24 chans x 32//(2*2*2)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.batch_norm_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.batch_norm_2(x)
        x = self.relu_2(x)
        x = self.pool_2(x)

        x = self.conv_3(x)
        x = self.batch_norm_3(x)
        x = self.relu_3(x)
        x = self.pool_3(x)

        x = x.view(-1, 24 * 4 * 4)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc1(x)
        return x


net = Net()